In [14]:
import sys, os
sys.path.append(os.path.abspath('..'))

In [15]:
import torch
from Data import GraphDataset
from Models import GCNFeatureExtractor, CLWrapper
from utils.train import train

In [16]:
use_pre_paired = True
batch_size = 3
lr = 0.01
epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_size = 10
pooling = 'global_avg'

In [17]:
# load data 
from Data import load_graphs

basic_data, _ = load_graphs()

	Collected 1 sample from null_ops.
	Collected 4 samples from commutations.
	Collected 7 samples from equivalences.
	Collected 7 samples from combined.
Loaded 19 samples and 19 quantum circuits from subset.


In [18]:
# import pickle

# def load_graphs(data_dir='../Data/raw/', file_name='handcrafted_dataset.pkl', subset=None):
#     file_path = os.path.join(data_dir, file_name)
#     graphs, qcs = [], []
    
#     with open(file_path, 'rb') as f:
#         dataset = pickle.load(f)

#     if file_name == 'handcrafted_dataset.pkl':
#         if subset is not None:
#             dataset = dataset[subset]
#             print(f"Loaded {len(dataset)} elements from subset {subset}:")
#         # collect all graphs in dataset, which are stored in a nested dictionary
#         dataset = collect_from_dict(dataset) 

#     # extract graphs and qcs separately, if needed
#     for sample in dataset:
#         if isinstance(sample, tuple): # if a tuple (qc, graph)
#             g, qc = sample
#             graphs.append(g)
#             qcs.append(qc)
#         elif all(isinstance(graph, tuple) for graph in sample):# if a list of tuples (qc, graph) 
#             g, qc = zip(*sample)
#             qcs.append(list(qc))
#             graphs.append(list(g))

#     print(f"Loaded {len(graphs)} samples and {len(qcs)} quantum circuits from subset.")
    
#     return graphs, qcs

# def collect_from_dict(dictionary):
#     graphs = []
#     for k, v in dictionary.items():
#         if isinstance(v, dict):
#             graphs.extend(collect_from_dict(v))
#         elif isinstance(v, list):
#             if all(isinstance(item, list) for item in v):  # if list of lists
#                 graphs.extend(v)
#                 print(f"\tCollected {len(v)} items from {k}.")
#             elif all(isinstance(item, tuple) for item in v):  # if list of tuples
#                 graphs.append(v)
#                 print(f"\tCollected 1 sample from {k}.")
                
#     return graphs

## Basic transforms training

In [19]:
dataset = GraphDataset(basic_data[:-1], pre_paired=use_pre_paired)

In [20]:
graphs = basic_data[0]
print(len(graphs))

18


In [21]:
import numpy as np
import networkx as nx
def get_attr_matrix(graph):
    nodes_list = list(graph.nodes)
    nodes_view = graph.nodes(data=True)
    return np.array([nodes_view[node]['feature_vector'] for node in nodes_list])

In [22]:
# Check if graphs in the dataset are correctly converted to PyTorch Geometric Data objects
graphs = [basic_data[0][0:2]]
dataset_1 = GraphDataset(graphs, pre_paired=use_pre_paired)

data_1, data_2 = dataset_1[0]

print("NODE FEATURES:")

print(get_attr_matrix(graphs[0][0]))
print(get_attr_matrix(graphs[0][1]))

print("---")

print(data_1.x)
print(data_2.x)

print("ADJACENCY:")

print(nx.adjacency_matrix(graphs[0][0]).todense())
print(nx.adjacency_matrix(graphs[0][1]).todense())

print("---")

print(data_1.edge_index)
print(data_2.edge_index)

NODE FEATURES:
[[0 1 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 0 0 1]]
[[0 1 0 0 0 0]
 [1 0 0 0 1 0]
 [1 0 0 0 0 1]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]]
---
tensor([[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.]])
ADJACENCY:
[[0 1 0]
 [0 0 1]
 [0 1 0]]
[[0 1 0 0 0]
 [0 0 1 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
---
tensor([[0, 1, 1, 2, 3],
        [1, 2, 3, 1, 4]])
tensor([[0, 1, 2],
        [1, 2, 1]])


In [23]:
dataset_1

In [24]:
# create a fully connected nn stacking fully connected layers and ReLU activations
out_dim_gnn = 64

fc_model = torch.nn.Sequential(
    torch.nn.Linear(out_dim_gnn, 256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256, 128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128, embedding_size)
)



In [25]:
n_features = dataset[0][0].x.shape[1] if use_pre_paired else dataset[0].x.shape[1]
print(f'Number of features in the dataset: {n_features}')

gnn = GCNFeatureExtractor(in_channels=n_features, out_channels=embedding_size, pooling_strategy=pooling)
proj = fc_model

# No projection head
model = CLWrapper(gnn).to(device)

Number of features in the dataset: 6


In [26]:
train(model, dataset, epochs, batch_size, tau= 1.0, device=device)

Epoch 1/100: 100%|██████████| 6/6 [00:00<00:00, 25.36batch/s, Loss=1.57]


Epoch 1/100 completed. Avg Loss: 1.5750


Epoch 2/100: 100%|██████████| 6/6 [00:00<00:00, 47.53batch/s, Loss=1.57]


Epoch 2/100 completed. Avg Loss: 1.5664


Epoch 3/100: 100%|██████████| 6/6 [00:00<00:00, 55.84batch/s, Loss=1.48]


...

Input representations:
 

Epoch 100/100:  67%|██████▋   | 4/6 [00:00<00:00, 34.39batch/s, Loss=1.18]

tensor([[ 1.0973e-01, -4.2434e-01,  3.2765e-01, -1.4701e-01, -3.7467e-01,
         -1.5895e-01,  3.2865e-01,  2.8424e-01,  2.6551e-01,  5.0397e-01],
        [ 6.1507e-01,  1.4024e-01, -3.0142e-01,  4.4778e-01, -3.5685e-01,
          2.4893e-01,  1.1806e-01, -1.7668e-01, -2.7579e-01, -1.1973e-02],
        [ 5.1013e-01,  1.9867e-01, -3.5249e-01,  4.4016e-01, -2.5609e-01,
          2.6141e-01,  4.6874e-02, -2.4684e-01, -3.9730e-01, -1.6560e-01],
        [ 2.3907e-01,  1.1930e-01, -4.9789e-01,  5.8901e-02, -1.6954e-01,
          3.0985e-01, -2.0877e-02, -4.4812e-03, -5.8746e-01, -4.5491e-01],
        [ 6.0522e-02, -1.8120e-01,  4.4700e-01,  1.2699e-01, -5.2287e-04,
         -3.2098e-01,  2.4908e-01, -1.9266e-02,  4.6238e-01,  6.0690e-01],
        [ 1.1142e-01,  1.9487e-01, -4.7025e-01,  1.9519e-01, -1.9101e-02,
          2.4697e-01, -1.3743e-01, -1.5969e-01, -5.7597e-01, -5.0289e-01]],
       grad_fn=<CatBackward0>)
Positive pairs scores:
 tensor([[-0.5753],
        [-0.2477],
        [ 0.

Epoch 100/100: 100%|██████████| 6/6 [00:00<00:00, 28.82batch/s, Loss=1.27]

Epoch 100/100 completed. Avg Loss: 1.2717
